# 目次
- [Guidanceについて](../../../../code/01.Introduce)
- [セットアップ](../../../../code/01.Introduce)
- [制約のない生成](../../../../code/01.Introduce)
- [Phi 3のための発言](../../../../code/01.Introduce)
- [正規表現](../../../../code/01.Introduce)
- [選択](../../../../code/01.Introduce)
- [思考の連鎖](../../../../code/01.Introduce)
- [JSON生成](../../../../code/01.Introduce)
- [HTML生成](../../../../code/01.Introduce)


# Guidanceについて
Guidanceは、あらゆる言語モデル（LM）の出力を制御するための実績あるオープンソースのPythonライブラリです。1回のAPI呼び出しで、モデルが従うべき正確なプログラム的制約をPythonで表現し、JSON、Python、HTML、SQL、またはユースケースが必要とする任意の構造で構造化された出力を生成することができます。

Guidanceは従来のプロンプト技術とは異なります。推論層でモデルをトークンごとに誘導することで制約を強制し、高品質な出力を生成します。また、高度に構造化されたシナリオで使用する場合、コストとレイテンシを30～50%削減することが可能です。

Guidanceについてさらに詳しく知りたい方は、[GitHubの公開リポジトリ](https://github.com/guidance-ai/guidance)をご覧いただくか、Microsoft Buildでの[Guidance Breakout Session](https://www.youtube.com/watch?v=qXMNPVVlCMs)をご視聴ください。


# セットアップ
1. Guidanceをインストールするには、`pip install guidance --pre` を実行してください。
2. AzureでPhi 3.5 miniエンドポイントをデプロイするには、https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml にアクセスし、「Deploy」ボタンをクリックしてください。
3. エンドポイントのAPIキーを`AZURE_PHI3_KEY`という環境変数に保存し、URLを`AZURE_PHI3_URL`という環境変数に保存してください。


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# 制約のない生成
`gen()` 関数を使用すると、制約なしでテキストを生成できます。これは、Guidanceを使用せずにモデルを使用する場合と同じです。

## チャットのフォーマット
多くのチャットモデルと同様に、Phi-3はユーザーとアシスタント間のメッセージを特定のフォーマットで期待します。GuidanceはPhi-3のチャットテンプレートをサポートしており、チャットのフォーマットを管理します。チャットのターンを作成するには、会話の各部分を`with user()`または`with assistant()`ブロックに配置します。`with system()`ブロックを使用してシステムメッセージを設定することもできます。


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## トークン節約
高度に構造化されたシナリオでは、Guidanceはトークンをスキップし、必要なトークンのみを生成することで、パフォーマンスを向上させ、効率を高め、APIコストを節約できます。生成されたトークンは、このノートブック内でハイライトされた背景で表示されます。強制されたトークンはハイライトなしで表示され、入力トークンと同じコストがかかります。入力トークンのコストは出力トークンの約3分の1と推定されています。

*注:* 制約のない生成を行った最初の例では、制約を提供しなかったため、トークンを強制することができませんでした。


# Phi 3について話す  
Guidanceを使用すると、モデルの応答に簡単にテキストを挿入することができます。これにより、モデルの出力を特定の方向に誘導したい場合に役立ちます。


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# 正規表現による制約
前の例では、Phi 3が`Canberra`という答えを出した後に追加の説明を行いました。モデルの出力を正確に1語に制約するためには、正規表現を使用することができます。


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

# 複数の選択肢から選ぶ

いくつかの選択肢が既知の場合、`select()` 関数を使用してモデルに選択肢のリストから選ばせることができます。


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

`select()` を使用すると、生成されたトークンは `Can` のみでした。`Canberra` が応答を完成させる唯一の選択肢であるため、残りのトークンは強制されました。


# 思考の連鎖
思考の連鎖は、問題を段階的に処理するよう促すことで、モデルの出力品質を向上させるためのテクニックです。通常、最終的な答えに到達するには、複数回のプロンプトが必要です。まず、モデルに段階的に考えるよう指示します。その後、再度プロンプトを与えて最終的な答えを提供させます。標準的なチャット推論APIを使用する場合、このプロセスには2回のAPI呼び出しが必要であり、モデルが生成した「思考の連鎖」は2回課金されます。一度目はモデルが生成した際の出力トークンとして、二度目は次の呼び出しの入力トークンとしてです。しかし、Guidanceを使用することで、全ての段階的なプロセスが単一のAPI呼び出しとして処理・課金されるため、コストと遅延が削減されます。


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# JSON生成
Guidanceを使用すると、ここで示されているユーザープロファイルスキーマのようなJSONスキーマやpydanticモデルに準拠したJSONを生成することができます。


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## HTML生成

Guidanceはコードを生成し、プログラミング言語の構文要件に従うこともできます。このセクションでは、非常にシンプルなHTMLウェブページを作成するための小さなGuidanceプログラムを作成します。

ウェブページを小さなセクションに分割し、それぞれに専用のGuidance関数を割り当てます。これらを最終的な関数で組み合わせてHTMLウェブページを作成します。
その後、この関数をAzure AIでGuidance対応モデルに対して実行します。

*注:* これは完全な機能を備えたHTMLジェネレーターではありません。目的は、個々のニーズに合わせた構造化された出力を作成する方法を示すことです。

まず、Guidanceから必要なものをインポートします:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTMLウェブページは非常に構造化されており、Guidanceを使用してページのその部分を「強制」します。  
モデルから明示的にテキストを要求する場合、タグとみなされる可能性のあるものを含めないようにする必要があります。つまり、'<'や'>'の文字を除外しなければなりません。


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

次に、この関数を使用して任意のHTMLタグ内にテキストを生成できます。


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

ページヘッダーを作成しましょう。  
その一環として、ページタイトルを生成する必要があります。


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

HTMLページの本文は、見出しや段落で構成されます。  
それぞれを作成するための関数を定義することができます。


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

さて、HTMLの本文を定義する関数です。  
これには `select()` を `recurse=True` として使用し、複数の見出しや段落を生成します:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

次に、完全なHTMLページを生成する関数について説明します。  
まずHTMLの開始タグを追加し、次にヘッダーを生成し、その後本文を作成し、最後にHTMLの終了タグを追加します。


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

私たちは使いやすいラッパーを提供します。これにより以下が可能になります：
- 生成の温度を設定する
- Modelオブジェクトから生成されたページを取得する


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

次に出力をファイルに書き込むことができます:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

そして[結果を見る](../../../../code/01.Introduce/sample_page.html)。



---

**免責事項**:  
この文書はAI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書を正式な情報源としてお考えください。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解について、当社は責任を負いません。
